In [2]:
# Import required libraries

# pandas - For data manipulation and analysis of tabular data (landmarks and protocol files)
import pandas as pd

# numpy - For numerical operations and array manipulation of pupil measurements
import numpy as np

# matplotlib - For creating static visualizations of pupil response data
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import to_rgba

# pathlib - For cross-platform 
# file path handling
from pathlib import Path

# logging - For tracking program execution and debugging information
import logging

# scipy.signal - For signal processing operations like Savitzky-Golay filtering
from scipy import signal

# seaborn - For enhanced statistical data visualization built on matplotlib
import seaborn as sns

# shapely.geometry - For polygon area calculation
from shapely.geometry import Polygon
from scipy.interpolate import CubicSpline # Add this import

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [3]:
# Modify cell 'ae6a1e82'

def load_validate_file(file_path):
    """
    Load and validate landmarks data file

    Args:
        file_path (Path): Path object to landmarks CSV file

    Returns:
        pd.DataFrame or None: Processed landmarks data, or None if loading/validation fails.
    """
    try:
        df = pd.read_csv(file_path)

        # Required columns for timestamp and status
        base_columns = ['timestamp', 'id', 'retcode']

        # Generate expected landmark column names for both eyes
        landmark_columns = []
        for eye in ['left', 'right']:
            for i in range(1, 28):  # 27 landmarks per eye
                landmark_columns.extend([
                    f'{eye}_lm_{i}_x',
                    f'{eye}_lm_{i}_y'
                ])

        required_columns = base_columns + landmark_columns

        # Verify all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            # Log warning instead of raising error immediately to allow processing other files
            logger.warning(f"File {file_path.name}: Missing required columns: {', '.join(missing_columns)}. Skipping file.")
            return None # Indicate failure

        logger.info(f"Successfully validated landmarks file {file_path.name} with {len(df)} rows")
        return df
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logger.error(f"Error loading or validating file {file_path.name}: {e}")
        return None # Indicate failure

# Define paths
data_path = Path().cwd().parent / "data"
output_path = Path().cwd().parent / "output"
output_path.mkdir(parents=True, exist_ok=True) # Ensure output directory exists

# Find all landmarks.csv files recursively
all_landmarks_files = list(data_path.glob('**/*_plr_landmarks.csv'))
logger.info(f"Found {len(all_landmarks_files)} landmark files to process.")

# Optional: Limit the number of files for testing
# all_landmarks_files = all_landmarks_files[:2]
# logger.info(f"Processing only the first {len(all_landmarks_files)} files for testing.")

# We will loop through these files in a subsequent cell

2025-04-17 19:50:31,789 - INFO - Found 55 landmark files to process.


# Pupil Size Extraction

In [ ]:
def get_pupil_boundary_points(frame, eye):
    """Extract pupil boundary points for a specific eye and frame"""
    boundary_points = []
    for lm in [7, 25, 9, 22, 10, 23, 24]:  # Pupil landmarks (not closing the loop with 7 again)
        x = frame[f'{eye}_lm_{lm}_x'].iloc[0]
        y = frame[f'{eye}_lm_{lm}_y'].iloc[0]
        boundary_points.append((x, y))
    return boundary_points

def get_pupil_center(frame, eye):
    """Get the center point (landmark 8) of the pupil"""
    center_x = frame[f'{eye}_lm_8_x'].iloc[0]
    center_y = frame[f'{eye}_lm_8_y'].iloc[0]
    return center_x, center_y

def cartesian_to_polar(center, boundary_points):
    """Convert Cartesian boundary points to polar coordinates relative to the center."""
    points_polar = []
    for p in boundary_points:
        dx = p[0] - center[0]
        dy = p[1] - center[1]
        angle = np.arctan2(dy, dx)
        distance = np.sqrt(dx**2 + dy**2)
        points_polar.append((angle, distance))
    return points_polar

def sort_polar_points(points_polar):
    """Sort polar points by angle."""
    points_polar.sort(key=lambda pp: pp[0])
    angles = [pp[0] for pp in points_polar]
    distances = [pp[1] for pp in points_polar]
    return angles, distances

def prepare_spline_data(angles, distances):
    """Prepare angles and distances for periodic spline interpolation."""
    if not angles: # Handle empty input
        logger.warning("prepare_spline_data received empty angles list.")
        return [], []
    if len(angles) != len(distances):
         logger.error("prepare_spline_data received angles and distances lists of different lengths.")
         return [], []

    # Create copies to avoid modifying original lists if they are reused elsewhere
    wrapped_angles = angles.copy()
    wrapped_distances = distances.copy()

    # Always wrap the angle and distance for periodic spline.
    # Append the first angle + 2pi and explicitly append the first distance
    # to ensure the endpoint matches the start point's distance value.
    wrapped_angles.append(wrapped_angles[0] + 2 * np.pi)
    wrapped_distances.append(wrapped_distances[0]) # Ensure y[0] == y[-1]

    # Ensure strictly increasing sequence for angles AFTER wrapping.
    # This is crucial for CubicSpline.
    eps = 1e-9 # A small epsilon value
    for i in range(1, len(wrapped_angles)):
        if wrapped_angles[i] <= wrapped_angles[i-1]:
            # Add a small epsilon, potentially scaled by the angle magnitude
            adjustment = eps * (1 + abs(wrapped_angles[i-1]))
            new_angle = wrapped_angles[i-1] + adjustment
            # Check if adjustment pushes the last angle too far
            if i == len(wrapped_angles) - 1 and new_angle > wrapped_angles[0] + 2 * np.pi:
                 # If the last point needs adjustment, clamp it to maintain periodicity
                 wrapped_angles[i] = wrapped_angles[0] + 2 * np.pi
                 logger.warning(f"Clamped last angle during epsilon adjustment to maintain periodicity.")
            else:
                 wrapped_angles[i] = new_angle


    # Final check: After adjustments, force the last angle to be exactly periodic if it drifted slightly.
    if not np.isclose(wrapped_angles[-1], wrapped_angles[0] + 2 * np.pi):
        logger.warning(f"Forcing last angle to ensure periodicity after epsilon adjustments. Diff: {wrapped_angles[-1] - (wrapped_angles[0] + 2 * np.pi)}")
        wrapped_angles[-1] = wrapped_angles[0] + 2 * np.pi

    # Ensure distances still match (should be guaranteed by the initial append, but safe to double-check)
    wrapped_distances[-1] = wrapped_distances[0]

    return wrapped_angles, wrapped_distances

def create_spline_interpolator(angles, distances):
    """Create a cubic spline interpolator using periodic boundary conditions."""
    if not angles or not distances: # Handle empty lists
        logger.warning("create_spline_interpolator received empty angles or distances.")
        return None # Or raise an error
    # Ensure the input data is suitable for periodic spline (last angle = first angle + 2*pi)
    # This is handled in prepare_spline_data
    if not np.isclose(angles[-1], angles[0] + 2 * np.pi):
         logger.warning("Input angles may not be properly wrapped for periodic spline.")
    # Use 'periodic' boundary condition
    return CubicSpline(angles, distances, bc_type='periodic')

def generate_interpolated_polar_points(cs, start_angle, num_points=50):
    """Generate smooth interpolated points in polar coordinates."""
    if cs is None:
        return [], []
    interp_angles = np.linspace(start_angle, start_angle + 2*np.pi, num_points, endpoint=False)
    interp_distances = cs(interp_angles)
    return interp_angles, interp_distances

def polar_to_cartesian(center, angles, distances):
    """Convert polar coordinates back to Cartesian coordinates."""
    interp_points = []
    for angle, dist in zip(angles, distances):
        # Normalize angle to be within [0, 2*pi) if needed, though linspace should handle it
        norm_angle = angle % (2 * np.pi)
        x = center[0] + dist * np.cos(norm_angle)
        y = center[1] + dist * np.sin(norm_angle)
        interp_points.append((x, y))
    return interp_points

def interpolate_pupil(center, boundary_points, num_points=36):
    """Create a smooth interpolated pupil boundary using cubic spline."""
    if not boundary_points:
        logger.warning("interpolate_pupil received empty boundary_points.")
        return [] # Return empty list if no boundary points

    # 1. Convert to polar coordinates
    points_polar = cartesian_to_polar(center, boundary_points)

    # 2. Sort by angle
    angles, distances = sort_polar_points(points_polar)

    # 3. Prepare data for spline (wrapping, ensure increasing angles)
    spline_angles, spline_distances = prepare_spline_data(angles, distances)

    # Handle case where preparation might fail (e.g., empty input)
    if not spline_angles:
        logger.warning("Could not prepare spline data, possibly due to insufficient input points.")
        return []

    # 4. Create spline interpolator
    cs = create_spline_interpolator(spline_angles, spline_distances)
    if cs is None:
        logger.warning("Failed to create spline interpolator.")
        return []


    # 5. Generate interpolated points in polar coordinates
    start_angle = min(spline_angles) # Use the minimum angle from the prepared data
    interp_angles, interp_distances = generate_interpolated_polar_points(cs, start_angle, num_points)

    # 6. Convert back to Cartesian coordinates
    interp_points = polar_to_cartesian(center, interp_angles, interp_distances)

    return interp_points


def get_eye_boundary_points(frame, eye):
    """Extract eye boundary points for a specific eye and frame."""
    boundary_points = []
    # Landmarks for the entire eye boundary: 1, 21, 12, 19, 14, 15, 5, 17, 4, 3, 2, 26
    eye_landmarks = [1, 21, 12, 19, 14, 15, 5, 17, 4, 3, 2, 26]
    for lm in eye_landmarks:
        try:
            x = frame[f'{eye}_lm_{lm}_x'].iloc[0]
            y = frame[f'{eye}_lm_{lm}_y'].iloc[0]
            boundary_points.append((x, y))
        except KeyError:
            logger.error(f"Landmark {lm} for {eye} eye not found in frame.")
            return [] # Return empty list if a landmark is missing
        except IndexError:
             logger.error(f"Index error accessing landmark {lm} for {eye} eye in frame.")
             return [] # Return empty list if frame data is inaccessible
    return boundary_points

def create_eye_polygon(boundary_points):
    """Create the eye polygon object from boundary points."""
    if len(boundary_points) < 3:
        logger.warning(f"Not enough points ({len(boundary_points)}) to form an eye polygon.")
        return None # Return None if not enough points
    try:
        # The list of points already defines the polygon vertices in order.
        polygon = Polygon(boundary_points)
        return polygon
    except Exception as e:
        logger.error(f"Error creating eye polygon: {e}")
        return None

def calculate_eyelid_distance(frame, eye):
    """Calculate the vertical distance between upper (3) and lower (19) eyelid landmarks."""
    try:
        upper_eyelid_x = frame[f'{eye}_lm_3_x'].iloc[0]
        upper_eyelid_y = frame[f'{eye}_lm_3_y'].iloc[0]
        lower_eyelid_x = frame[f'{eye}_lm_19_x'].iloc[0]
        lower_eyelid_y = frame[f'{eye}_lm_19_y'].iloc[0]

        # Calculate Euclidean distance
        distance = np.sqrt((upper_eyelid_x - lower_eyelid_x)**2 + (upper_eyelid_y - lower_eyelid_y)**2)
        return distance
    except KeyError as e:
        logger.error(f"Eyelid landmark missing for {eye} eye: {e}")
        return np.nan
    except IndexError:
        logger.error(f"Index error accessing eyelid landmarks for {eye} eye.")
        return np.nan
    except Exception as e:
        logger.error(f"Error calculating eyelid distance for {eye} eye: {e}")
        return np.nan

### Now we run this logic into all of the frames of a test

In [5]:
# New cell after cell '20abba5d'

def process_landmarks_file(landmarks_df_cleaned):
    """
    Processes a cleaned landmarks DataFrame to calculate pupil areas,
    eyelid distances, and generate polygon data.

    Args:
        landmarks_df_cleaned (pd.DataFrame): DataFrame containing cleaned landmark data
                                             for a single recording, filtered for 'OK' retcode
                                             and indexed correctly.

    Returns:
        tuple: Contains:
            - pd.DataFrame: The input DataFrame augmented with calculated columns.
            - dict: Dictionary of interpolated points for the right eye {frame_index: points_list}.
            - dict: Dictionary of interpolated points for the left eye {frame_index: points_list}.
            - dict: Dictionary of eye polygon objects for the right eye {frame_index: Polygon}.
            - dict: Dictionary of eye polygon objects for the left eye {frame_index: Polygon}.
    """
    logger.info(f"Processing {len(landmarks_df_cleaned)} cleaned frames...")

    # Dictionary to store results for both eyes
    eye_data = {
        'right': {
            'results_list': [],
            'interpolated_points_dict': {},
            'eye_polygons_dict': {}
        },
        'left': {
            'results_list': [],
            'interpolated_points_dict': {},
            'eye_polygons_dict': {}
        }
    }

    # Process both eyes
    for eye in ['right', 'left']:
        logger.info(f"Calculating metrics for {eye} eye...")
        # Iterate through each row (frame) of the cleaned DataFrame
        for index, frame_row in landmarks_df_cleaned.iterrows():
            # Create a DataFrame containing only the current row
            frame_df = landmarks_df_cleaned.iloc[[index]]
            timestamp = frame_row['timestamp']  # Get timestamp for storage

            # Initialize results for this frame
            interpolated_area = np.nan
            non_interpolated_area = np.nan
            eyelid_dist = np.nan
            eye_polygon_obj = None

            try:
                # --- Pupil Calculations ---
                center_x, center_y = get_pupil_center(frame_df, eye)
                center = (center_x, center_y)
                pupil_boundary_points = get_pupil_boundary_points(frame_df, eye)
                interpolated_pupil_points = interpolate_pupil(center, pupil_boundary_points, num_points=50)

                if len(interpolated_pupil_points) >= 3:
                     interpolated_polygon = Polygon(interpolated_pupil_points)
                     interpolated_area = interpolated_polygon.area
                else:
                     logger.debug(f"Not enough interpolated points ({len(interpolated_pupil_points)}) for pupil polygon for {eye} eye, frame index {index}.")

                if len(pupil_boundary_points) >= 3:
                    non_interpolated_polygon_points = pupil_boundary_points + [pupil_boundary_points[0]]
                    non_interpolated_polygon = Polygon(non_interpolated_polygon_points)
                    non_interpolated_area = non_interpolated_polygon.area
                else:
                    logger.debug(f"Not enough boundary points ({len(pupil_boundary_points)}) for non-interp pupil polygon for {eye} eye, frame index {index}.")

                # --- Eye and Eyelid Calculations ---
                eye_boundary_points = get_eye_boundary_points(frame_df, eye)
                if eye_boundary_points:
                    eye_polygon_obj = create_eye_polygon(eye_boundary_points)

                eyelid_dist = calculate_eyelid_distance(frame_df, eye)

                # --- Store Results ---
                eye_data[eye]['results_list'].append({
                    'frame_index': index,
                    'timestamp': timestamp,
                    f'{eye}_interpolated_area': interpolated_area,
                    f'{eye}_non_interpolated_area': non_interpolated_area,
                    f'{eye}_eyelid_distance': eyelid_dist,
                })

                if not np.isnan(interpolated_area):
                     eye_data[eye]['interpolated_points_dict'][index] = interpolated_pupil_points

                if eye_polygon_obj is not None:
                     eye_data[eye]['eye_polygons_dict'][index] = eye_polygon_obj

            except Exception as e:
                logger.error(f"Error processing {eye} eye for frame index {index} (Timestamp: {timestamp}): {e}", exc_info=True)
                # Store NaN entry in case of unexpected error
                eye_data[eye]['results_list'].append({
                    'frame_index': index,
                    'timestamp': timestamp,
                    f'{eye}_interpolated_area': np.nan,
                    f'{eye}_non_interpolated_area': np.nan,
                    f'{eye}_eyelid_distance': np.nan,
                })

    # --- After the loop ---
    # Create DataFrames from the collected results
    right_results_df = pd.DataFrame(eye_data['right']['results_list'])
    left_results_df = pd.DataFrame(eye_data['left']['results_list'])

    # Set index for merging if lists are not empty
    if not right_results_df.empty:
        right_results_df.set_index('frame_index', inplace=True)
    if not left_results_df.empty:
        left_results_df.set_index('frame_index', inplace=True)

    # Define columns to merge
    left_merge_cols = [col for col in left_results_df.columns if col not in ['timestamp', 'frame_index']]
    right_merge_cols = [col for col in right_results_df.columns if col != 'frame_index'] # Keep timestamp from right

    # Combine both eye results using outer merge on index
    # Start with an empty DataFrame or one of the results to handle cases where one eye might have no data
    if not right_results_df.empty:
        combined_results_df = right_results_df[right_merge_cols]
        if not left_results_df.empty:
            combined_results_df = combined_results_df.merge(
                left_results_df[left_merge_cols],
                left_index=True,
                right_index=True,
                how='outer'
            )
    elif not left_results_df.empty:
        # If right is empty but left is not, use left as the base
        # Need to rename timestamp column if it exists in left_results_df
        if 'timestamp' in left_results_df.columns:
             left_results_df.rename(columns={'timestamp': 'timestamp_left'}, inplace=True) # Avoid conflict if merging later
             left_merge_cols = [col for col in left_results_df.columns if col != 'frame_index'] # Update merge cols
        combined_results_df = left_results_df[left_merge_cols]
    else:
        # If both are empty, create an empty DataFrame with expected index
        combined_results_df = pd.DataFrame(index=landmarks_df_cleaned.index)


    # --- Merge back into the main DataFrame ---
    calculated_columns_to_merge = [col for col in combined_results_df.columns if col != 'timestamp']
    available_cols = [col for col in calculated_columns_to_merge if col in combined_results_df.columns]

    # Ensure 'timestamp' column exists in combined_results_df before trying to merge it if needed
    if 'timestamp' in combined_results_df.columns and 'timestamp' not in landmarks_df_cleaned.columns:
         # If timestamp isn't already in cleaned_df, include it from combined
         available_cols.append('timestamp')


    landmarks_df_calculated = landmarks_df_cleaned.merge(
        combined_results_df[available_cols],
        left_index=True,
        right_index=True,
        how='left'
    )

    # Convert timestamp string to datetime objects if the column exists
    if 'timestamp' in landmarks_df_calculated.columns:
        landmarks_df_calculated['timestamp_dt'] = pd.to_datetime(landmarks_df_calculated['timestamp'], errors='coerce')
    else:
        logger.warning("Timestamp column not found after merge, cannot create 'timestamp_dt'.")


    # Store interpolated points and eye polygons dictionaries for later access
    right_interpolated_points_dict = eye_data['right']['interpolated_points_dict']
    left_interpolated_points_dict = eye_data['left']['interpolated_points_dict']
    right_eye_polygons_dict = eye_data['right']['eye_polygons_dict']
    left_eye_polygons_dict = eye_data['left']['eye_polygons_dict']

    logger.info("Finished processing file.")
    return (landmarks_df_calculated,
            right_interpolated_points_dict, left_interpolated_points_dict,
            right_eye_polygons_dict, left_eye_polygons_dict)

In [7]:
# Insert this code in a new cell before cell '538ec641' (the main processing loop)
# or add it to cell '4cd9c6e2' after the 'load_validate_file' function.

def filter_ok_landmarks_data(df):
    """
    Filters the landmarks DataFrame to keep only rows where 'retcode' is 'OK'.

    Args:
        df (pd.DataFrame): The input landmarks DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows with 'retcode' == 'OK'.
                      Returns an empty DataFrame if the input is None, empty,
                      or if the 'retcode' column is missing.
    """
    if df is None or df.empty:
        logger.warning("filter_ok_landmarks_data received an empty or None DataFrame.")
        return pd.DataFrame() # Return an empty DataFrame

    if 'retcode' not in df.columns:
        logger.error("filter_ok_landmarks_data: 'retcode' column not found in DataFrame.")
        return pd.DataFrame() # Return an empty DataFrame

    try:
        # Filter rows where retcode is 'OK'
        filtered_df = df[df['retcode'] == 'OK'].copy() # Use .copy() to avoid SettingWithCopyWarning
        logger.info(f"Filtered data: {len(filtered_df)} frames with 'OK' retcode remaining (out of {len(df)}).")

        # Optional: Reset index if needed, though processing loop uses .iterrows() which is fine
        # filtered_df.reset_index(drop=True, inplace=True)

        return filtered_df
    except Exception as e:
        logger.error(f"Error filtering DataFrame by 'retcode': {e}")
        return pd.DataFrame() # Return an empty DataFrame in case of other errors

In [8]:
# In cell with id '629c936e'

# --- Animation with Trail, More Modular Structure ---
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display # Ensure display is imported
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import deque
from matplotlib.colors import to_rgba, LinearSegmentedColormap
from pathlib import Path
import matplotlib.patches as patches # Import patches for legend
import logging # Make sure logging is available

logger = logging.getLogger(__name__) # Ensure logger is defined if not already

# --- Helper functions for animation (Refactored to accept data) ---

def prepare_data_for_animation(landmarks_df_calculated, right_interpolated_points_dict, left_interpolated_points_dict, max_frames=None):
    """
    Prepare timestamp data and find valid frames for animation for both eyes.
    Accepts data as arguments.
    """
    # Create a copy to avoid modifying the original DataFrame passed to the function
    df_calc = landmarks_df_calculated.copy()

    # Ensure timestamp_dt exists and calculate time_elapsed_s
    if 'timestamp_dt' not in df_calc.columns or not pd.api.types.is_datetime64_any_dtype(df_calc['timestamp_dt']):
        if 'timestamp' in df_calc.columns:
            df_calc['timestamp_dt'] = pd.to_datetime(df_calc['timestamp'], errors='coerce')
        else:
            logger.error("Cannot prepare animation data: 'timestamp' column missing.")
            return None, None # Return None for both df and valid_frames

    first_timestamp = df_calc['timestamp_dt'].dropna().min()
    if pd.isna(first_timestamp):
        logger.warning("Could not determine the first valid timestamp for animation timing.")
        df_calc['time_elapsed_s'] = np.nan
    else:
        df_calc['time_elapsed_s'] = (df_calc['timestamp_dt'] - first_timestamp).dt.total_seconds()

    # Filter valid frames based on *either* eye having valid data and existing time
    all_indices = set(right_interpolated_points_dict.keys()) | set(left_interpolated_points_dict.keys())
    # Ensure index exists in df_calc before checking time_elapsed_s
    valid_indices_in_df = df_calc.index.intersection(all_indices)

    valid_frames_all = sorted([
        idx for idx in valid_indices_in_df
        if not pd.isna(df_calc.loc[idx, 'time_elapsed_s']) and \
           ((idx in right_interpolated_points_dict and not pd.isna(df_calc.loc[idx, 'right_interpolated_area'])) or \
            (idx in left_interpolated_points_dict and not pd.isna(df_calc.loc[idx, 'left_interpolated_area'])))
    ])

    # Limit the number of frames if max_frames is specified
    if max_frames is not None:
        valid_frames = valid_frames_all[:max_frames]
        limit_msg = f"(limited to {max_frames} from {len(valid_frames_all)})"
    else:
        valid_frames = valid_frames_all
        limit_msg = ""

    if not valid_frames:
        logger.warning("No valid frames found to animate for either eye.")
        return df_calc, None # Return df_calc (with time_elapsed_s) and None for valid_frames

    logger.info(f"Using {len(valid_frames)} frames for animation {limit_msg}.")

    # Return the modified DataFrame and the list of valid frame indices
    return df_calc, valid_frames

def calculate_plot_limits_for_animation(valid_frames, right_interpolated_points_dict, left_interpolated_points_dict, right_eye_polygons_dict, left_eye_polygons_dict):
    """
    Calculate plot limits based on all valid points (pupil + eye outlines) for both eyes.
    Accepts data as arguments.
    """
    all_x = []
    all_y = []

    # Include pupil points
    for eye_dict in [right_interpolated_points_dict, left_interpolated_points_dict]:
        for idx in valid_frames:
            if idx in eye_dict:
                points = eye_dict.get(idx, [])
                if points:
                    all_x.extend([p[0] for p in points])
                    all_y.extend([p[1] for p in points])

    # Include eye outline points
    for eye_poly_dict in [right_eye_polygons_dict, left_eye_polygons_dict]:
         for idx in valid_frames:
             if idx in eye_poly_dict:
                 poly = eye_poly_dict.get(idx)
                 if poly and hasattr(poly, 'exterior'):
                     coords = list(poly.exterior.coords)
                     all_x.extend([c[0] for c in coords])
                     all_y.extend([c[1] for c in coords])


    if not all_x or not all_y:
        logger.warning("No points found to determine animation limits.")
        return None, None, None, None

    # Use a fixed padding or percentage, ensure it handles single points
    x_min_val, x_max_val = min(all_x), max(all_x)
    y_min_val, y_max_val = min(all_y), max(all_y)

    x_range = x_max_val - x_min_val
    y_range = y_max_val - y_min_val

    # Use 30% padding as before, ensure non-zero padding if range is zero
    x_pad = x_range * 0.30 if x_range > 1e-6 else 20 # Add fixed padding if range is tiny
    y_pad = y_range * 0.30 if y_range > 1e-6 else 20 # Add fixed padding if range is tiny

    x_min = x_min_val - x_pad
    x_max = x_max_val + x_pad
    y_min = y_min_val - y_pad
    y_max = y_max_val + y_pad

    return x_min, x_max, y_min, y_max


def initialize_plot_for_animation(x_min, x_max, y_min, y_max, trail_length):
    """Initialize the figure, axes and plot elements for both eyes. (No data needed here)"""
    # Keep the larger figure size
    fig, ax = plt.subplots(figsize=(18, 10))

    # Define colors for each eye, adding eyelid line color
    colors = {
        'right': {'interp': 'red', 'boundary': 'blue', 'center_trail': (0.5, 0, 0.5), 'eye_poly': 'darkgreen', 'eyelid': 'magenta'},
        'left': {'interp': 'orange', 'boundary': 'cyan', 'center_trail': (0, 0.5, 0.5), 'eye_poly': 'darkred', 'eyelid': 'gold'}
    }

    plot_elements = {'eyes': {}, 'info_text': None, 'ax': ax, 'fig': fig} # Store fig too

    for eye in ['right', 'left']:
        eye_colors = colors[eye]
        plot_elements['eyes'][eye] = {}
        plot_elements['eyes'][eye]['interp_poly'] = plt.Polygon(
            [[0, 0]], color=eye_colors['interp'], alpha=0.3, label=f'{eye.capitalize()} Pupil', zorder=10)
        plot_elements['eyes'][eye]['boundary_poly'], = plt.plot(
            [], [], color=eye_colors['boundary'], linestyle='-', linewidth=1, alpha=0.5, label=f'{eye.capitalize()} Pupil Boundary', zorder=20)
        plot_elements['eyes'][eye]['center_trail_scatter'] = plt.scatter(
            [], [], s=50, facecolors='none', label=f'{eye.capitalize()} Center Trail',
            zorder=30 + (0 if eye == 'right' else 1), edgecolors=to_rgba(eye_colors['center_trail'], alpha=1.0), linewidths=1)
        plot_elements['eyes'][eye]['eye_poly'] = plt.Polygon(
            [[0, 0]], fill=False, edgecolor=eye_colors['eye_poly'], linewidth=1.5, alpha=0.7, label=f'{eye.capitalize()} Eye Outline', zorder=5)
        plot_elements['eyes'][eye]['eyelid_line'], = plt.plot(
            [], [], color=eye_colors['eyelid'], linestyle='--', linewidth=2, alpha=0.8, label=f'{eye.capitalize()} Eyelid Dist.', zorder=25)

        ax.add_patch(plot_elements['eyes'][eye]['interp_poly'])
        ax.add_patch(plot_elements['eyes'][eye]['eye_poly'])
        plot_elements['eyes'][eye]['center_history'] = deque(maxlen=trail_length)
        plot_elements['eyes'][eye]['base_color'] = eye_colors['center_trail']

    # Expand axes limits
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_max, y_min) # Remember y-axis is inverted

    # Info text
    plot_elements['info_text'] = plt.text(
        0.02, 0.02, '', transform=ax.transAxes, verticalalignment='bottom', horizontalalignment='left',
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8), zorder=50)

    ax.set_xlabel('X Coordinate (pixels)', fontsize=12)
    ax.set_ylabel('Y Coordinate (pixels)', fontsize=12)
    ax.grid(alpha=0.3)
    ax.set_aspect('equal')

    # Create custom legend handles
    legend_elements = []
    for eye in ['right', 'left']:
         legend_elements.extend([
             patches.Patch(facecolor=colors[eye]['interp'], alpha=0.5, label=f'{eye.capitalize()} Pupil'),
             plt.Line2D([0], [0], color=colors[eye]['boundary'], lw=1, label=f'{eye.capitalize()} Pupil Boundary'),
             plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor=colors[eye]['center_trail'], markersize=8, label=f'{eye.capitalize()} Center Trail'),
             patches.Patch(facecolor='none', edgecolor=colors[eye]['eye_poly'], linewidth=1.5, label=f'{eye.capitalize()} Eye Outline'),
             plt.Line2D([0], [0], color=colors[eye]['eyelid'], linestyle='--', lw=2, label=f'{eye.capitalize()} Eyelid Dist.')
         ])

    ax.legend(handles=legend_elements, loc='lower right', fontsize='small', framealpha=0.8, ncol=2)

    return fig, ax, plot_elements


def create_animation_func(fig, ax, plot_elements, valid_frames, frames_count,
                           landmarks_df_calculated, landmarks_df_cleaned,
                           right_interpolated_points_dict, left_interpolated_points_dict,
                           right_eye_polygons_dict, left_eye_polygons_dict):
    """
    Create the animation using FuncAnimation for both eyes.
    Accepts data as arguments.
    """
    info_text = plot_elements['info_text']

    # Helper functions (get_pupil_center, get_pupil_boundary_points) are assumed to be defined globally or imported
    # If they are not, they need to be passed or defined here as well.

    def init():
        # (init function remains largely the same, it just resets plot elements)
        elements_to_return = []
        for eye in ['right', 'left']:
            eye_elems = plot_elements['eyes'][eye]
            eye_elems['interp_poly'].set_xy([[0, 0]])
            eye_elems['boundary_poly'].set_data([], [])
            eye_elems['center_trail_scatter'].set_offsets(np.empty((0, 2)))
            eye_elems['center_trail_scatter'].set_edgecolors(np.empty((0, 4)))
            eye_elems['center_trail_scatter'].set_facecolors('none')
            eye_elems['eye_poly'].set_xy([[0, 0]])
            eye_elems['eyelid_line'].set_data([], [])
            eye_elems['center_history'].clear()
            elements_to_return.extend([
                eye_elems['interp_poly'], eye_elems['boundary_poly'],
                eye_elems['center_trail_scatter'], eye_elems['eye_poly'],
                eye_elems['eyelid_line']
            ])
        info_text.set_text('')
        elements_to_return.append(info_text)
        return tuple(elements_to_return)

    def update(frame_idx):
        # Access data passed to create_animation_func
        idx = valid_frames[frame_idx]

        # Check if index exists in DataFrames before accessing
        if idx not in landmarks_df_cleaned.index or idx not in landmarks_df_calculated.index:
             logger.warning(f"Index {idx} not found in DataFrames for frame {frame_idx}. Skipping update.")
             # Return the existing elements without updating them
             elements_to_return = []
             for eye in ['right', 'left']:
                  eye_elems = plot_elements['eyes'][eye]
                  elements_to_return.extend([
                      eye_elems['interp_poly'], eye_elems['boundary_poly'],
                      eye_elems['center_trail_scatter'], eye_elems['eye_poly'],
                      eye_elems['eyelid_line']
                  ])
             elements_to_return.append(info_text)
             return tuple(elements_to_return)


        frame_row = landmarks_df_cleaned.loc[idx]
        frame_df = landmarks_df_cleaned.loc[[idx]] # Use loc for single row DataFrame

        elements_to_return = []
        info_lines = [f"Frame Index: {idx}"]
        time_elapsed = landmarks_df_calculated.loc[idx, 'time_elapsed_s']
        info_lines.append(f"Time: {time_elapsed:.3f}s" if not pd.isna(time_elapsed) else "Time: NaN")

        for eye in ['right', 'left']:
            eye_elems = plot_elements['eyes'][eye]
            pupil_dict = right_interpolated_points_dict if eye == 'right' else left_interpolated_points_dict
            eye_poly_dict = right_eye_polygons_dict if eye == 'right' else left_eye_polygons_dict
            area_col = f'{eye}_interpolated_area'
            eyelid_dist_col = f'{eye}_eyelid_distance'

            # --- Update Eye Polygon ---
            eye_polygon_obj = eye_poly_dict.get(idx)
            if eye_polygon_obj and hasattr(eye_polygon_obj, 'exterior'):
                eye_poly_coords = list(eye_polygon_obj.exterior.coords)
                eye_elems['eye_poly'].set_xy(eye_poly_coords)
            else:
                eye_elems['eye_poly'].set_xy([[0, 0]])

            # --- Update Eyelid Distance Line ---
            eyelid_dist_val = landmarks_df_calculated.loc[idx, eyelid_dist_col]
            if not pd.isna(eyelid_dist_val):
                try:
                    upper_x = frame_row[f'{eye}_lm_3_x']
                    upper_y = frame_row[f'{eye}_lm_3_y']
                    lower_x = frame_row[f'{eye}_lm_19_x']
                    lower_y = frame_row[f'{eye}_lm_19_y']
                    eye_elems['eyelid_line'].set_data([upper_x, lower_x], [upper_y, lower_y])
                    info_lines.append(f"{eye.capitalize()} Eyelid Dist: {eyelid_dist_val:.2f} px")
                except KeyError:
                    eye_elems['eyelid_line'].set_data([], [])
                    info_lines.append(f"{eye.capitalize()} Eyelid Dist: Error (Key)")
            else:
                eye_elems['eyelid_line'].set_data([], [])
                info_lines.append(f"{eye.capitalize()} Eyelid Dist: N/A")

            # --- Update Pupil and Center Trail ---
            if idx in pupil_dict and not pd.isna(landmarks_df_calculated.loc[idx, area_col]):
                try:
                    center_x, center_y = get_pupil_center(frame_df, eye)
                    boundary_pts = get_pupil_boundary_points(frame_df, eye)
                    boundary_poly_pts = boundary_pts + [boundary_pts[0]] if len(boundary_pts) >= 1 else []
                    interp_pts = pupil_dict[idx]
                    interp_area = landmarks_df_calculated.loc[idx, area_col]

                    eye_elems['center_history'].append((center_x, center_y))
                    eye_elems['interp_poly'].set_xy(interp_pts if len(interp_pts) >= 3 else [[0,0]])
                    eye_elems['boundary_poly'].set_data([p[0] for p in boundary_poly_pts], [p[1] for p in boundary_poly_pts])
                    update_center_trail(eye_elems['center_history'], eye_elems['center_trail_scatter'], eye_elems['base_color'])
                    info_lines.append(f"{eye.capitalize()} Pupil Area: {interp_area:.2f} px²")
                except Exception as e:
                     logger.warning(f"Error updating pupil for {eye} eye, index {idx}: {e}")
                     eye_elems['interp_poly'].set_xy([[0,0]])
                     eye_elems['boundary_poly'].set_data([], [])
                     info_lines.append(f"{eye.capitalize()} Pupil Area: Error")

            else:
                eye_elems['interp_poly'].set_xy([[0,0]])
                eye_elems['boundary_poly'].set_data([], [])
                info_lines.append(f"{eye.capitalize()} Pupil Area: N/A")

            elements_to_return.extend([
                eye_elems['interp_poly'], eye_elems['boundary_poly'],
                eye_elems['center_trail_scatter'], eye_elems['eye_poly'],
                eye_elems['eyelid_line']
            ])

        ax.set_title(f'Left & Right Eye Pupils, Outlines & Eyelid Distance - Frame {frame_idx+1}/{len(valid_frames)} (Index: {idx})', fontsize=14)
        info_text.set_text('\n'.join(info_lines))
        elements_to_return.append(info_text)

        return tuple(elements_to_return)

    # Create the animation
    ani = FuncAnimation(
        fig, update, frames=frames_count, init_func=init,
        blit=False, interval=100, repeat=False # Set repeat=False for saving
    )

    return ani

def update_center_trail(center_history, center_trail_scatter, base_color):
    """Update the center trail scatter plot. (No changes needed here)"""
    num_points = len(center_history)
    if num_points > 0:
        center_trail_offsets = np.array(list(center_history))
        min_alpha = 0.1
        max_alpha = 1.0
        edge_colors = []
        for i in range(num_points):
            alpha = min_alpha + (max_alpha - min_alpha) * (i / (num_points - 1)) if num_points > 1 else max_alpha
            edge_colors.append((*base_color, alpha))
        center_trail_scatter.set_offsets(center_trail_offsets)
        center_trail_scatter.set_edgecolors(edge_colors)
        center_trail_scatter.set_facecolors('none')
    else:
        center_trail_scatter.set_offsets(np.empty((0, 2)))
        center_trail_scatter.set_edgecolors(np.empty((0, 4)))
        center_trail_scatter.set_facecolors('none')

def save_animation(ani, save_path):
    """Save the animation to the specified path. (No changes needed here)"""
    try:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        ani.save(str(save_path), writer='pillow', fps=10)
        logger.info(f"Animation saved to {save_path}")
    except Exception as e:
        logger.error(f"Error saving animation to {save_path}: {e}", exc_info=True)


# --- Main Animation Creation Function (Refactored) ---
def generate_and_save_animation(
    landmarks_df_calculated, landmarks_df_cleaned,
    right_interpolated_points_dict, left_interpolated_points_dict,
    right_eye_polygons_dict, left_eye_polygons_dict,
    save_path, trail_length=15, max_frames=None):
    """
    Orchestrates the creation and saving of the pupil animation for a single file's data.
    """
    logger.info(f"Starting animation generation for: {save_path.name}")

    # --- Step 1: Data Preparation ---
    df_calc_timed, valid_frames = prepare_data_for_animation(
        landmarks_df_calculated, right_interpolated_points_dict, left_interpolated_points_dict, max_frames
    )
    if valid_frames is None:
        logger.error("Animation generation failed: Could not prepare valid frames.")
        return None

    # --- Step 2: Set up plot limits ---
    x_min, x_max, y_min, y_max = calculate_plot_limits_for_animation(
        valid_frames, right_interpolated_points_dict, left_interpolated_points_dict,
        right_eye_polygons_dict, left_eye_polygons_dict
    )
    if x_min is None:
        logger.error("Animation generation failed: Could not calculate plot limits.")
        return None

    # --- Step 3: Create figure and initialize plot elements ---
    fig, ax, plot_elements = initialize_plot_for_animation(x_min, x_max, y_min, y_max, trail_length)

    # --- Step 4: Set up animation ---
    frames_count = len(valid_frames)
    try:
        # Pass all required data to the animation function
        ani = create_animation_func(
            fig, ax, plot_elements, valid_frames, frames_count,
            df_calc_timed, landmarks_df_cleaned, # Pass the timed df and the original cleaned df
            right_interpolated_points_dict, left_interpolated_points_dict,
            right_eye_polygons_dict, left_eye_polygons_dict
        )
    except Exception as e:
        logger.error(f"Error during FuncAnimation creation: {e}", exc_info=True)
        plt.close(fig) # Close the figure if animation creation fails
        return None


    # --- Step 5: Save ---
    save_animation(ani, save_path)

    # --- Step 6: Clean up ---
    plt.close(fig) # Close the figure to free memory
    logger.info(f"Finished animation generation for: {save_path.name}")
    return ani # Return the animation object if needed, though saving is the main goal


# filepath: /home/lrn/Repos/analyze-eye-tracking-data/notebooks/process_all_tests.ipynb
# In cell with id '4be5a042'

# ... (previous code in the cell remains the same) ...

# Dictionary to store results per file if needed later
all_results = {}

for landmarks_file_path in all_landmarks_files:
    logger.info(f"--- Processing file: {landmarks_file_path.name} ---")

    # 1. Load and Validate
    landmarks_df = load_validate_file(landmarks_file_path)
    if landmarks_df is None:
        logger.warning(f"Skipping file {landmarks_file_path.name} due to loading/validation errors.")
        continue # Skip to the next file

    # 2. Clean Data (Filter OK retcode)
    # Make sure filter_ok_landmarks_data is defined in a previous cell
    try:
        landmarks_df_cleaned = filter_ok_landmarks_data(landmarks_df)
    except NameError:
        logger.error("Function 'filter_ok_landmarks_data' not defined. Please define it in a previous cell.")
        break # Stop processing if essential function is missing
    except Exception as e:
         logger.error(f"Error cleaning data for {landmarks_file_path.name}: {e}")
         continue


    if landmarks_df_cleaned.empty:
        logger.warning(f"Skipping file {landmarks_file_path.name} as no 'OK' frames were found.")
        continue

    # 3. Process the cleaned data
    try:
        (landmarks_df_calculated,
         right_interpolated_points_dict, left_interpolated_points_dict,
         right_eye_polygons_dict, left_eye_polygons_dict) = process_landmarks_file(landmarks_df_cleaned)

        # --- Optional: Save calculated data ---
        output_csv_filename = landmarks_file_path.stem.replace('_plr_landmarks', '_processed_results') + ".csv"
        output_csv_path = output_path / output_csv_filename
        landmarks_df_calculated.to_csv(output_csv_path, index=False)
        logger.info(f"Saved calculated data to {output_csv_path}")

        # --- Optional: Store results for later use ---
        file_key = landmarks_file_path.stem
        all_results[file_key] = {
            'calculated_df': landmarks_df_calculated,
            'right_interp_pts': right_interpolated_points_dict,
            'left_interp_pts': left_interpolated_points_dict,
            'right_eye_polys': right_eye_polygons_dict,
            'left_eye_polys': left_eye_polygons_dict,
            'original_cleaned_df': landmarks_df_cleaned # Keep cleaned df for animation
        }

        # --- Generate and Save Animation ---
        # Check if data needed for animation exists before proceeding
        if not landmarks_df_calculated.empty and (right_interpolated_points_dict or left_interpolated_points_dict):
            animation_save_filename = file_key.replace('_plr_landmarks', '_animation') + ".gif"
            animation_save_path = output_path / animation_save_filename

            # Call the main animation function from cell '629c936e'
            generate_and_save_animation(
                landmarks_df_calculated=landmarks_df_calculated,
                landmarks_df_cleaned=landmarks_df_cleaned, # Pass the original cleaned df
                right_interpolated_points_dict=right_interpolated_points_dict,
                left_interpolated_points_dict=left_interpolated_points_dict,
                right_eye_polygons_dict=right_eye_polygons_dict,
                left_eye_polygons_dict=left_eye_polygons_dict,
                save_path=animation_save_path,
                trail_length=15,
                max_frames=None # Set max_frames=None to process all valid frames, or set a number e.g., 210
            )
        else:
            logger.warning(f"Skipping animation for {landmarks_file_path.name} due to missing calculated data or points.")


        logger.info(f"--- Finished processing file: {landmarks_file_path.name} ---")


    except Exception as e:
        logger.error(f"An unexpected error occurred while processing file {landmarks_file_path.name}: {e}", exc_info=True)
        # Continue to the next file

logger.info("=== All files processed ===")

# Optional: Display results summary or paths to generated files
print("\n--- Processing Summary ---")
print(f"Processed {len(all_results)} files.")
print("Generated files are located in:", output_path)
for key in all_results.keys():
    csv_path = output_path / (key.replace('_plr_landmarks', '_processed_results') + ".csv")
    gif_path = output_path / (key.replace('_plr_landmarks', '_animation') + ".gif")
    print(f" - {key}:")
    print(f"   - CSV: {csv_path.name}")
    print(f"   - GIF: {gif_path.name}")

2025-04-17 19:52:13,323 - INFO - --- Processing file: 60f931f1-b263-4947-80e6-18869129b673_plr_landmarks.csv ---
2025-04-17 19:52:13,339 - INFO - Successfully validated landmarks file 60f931f1-b263-4947-80e6-18869129b673_plr_landmarks.csv with 267 rows
2025-04-17 19:52:13,342 - INFO - Filtered data: 267 frames with 'OK' retcode remaining (out of 267).
2025-04-17 19:52:13,345 - INFO - Processing 267 cleaned frames...
2025-04-17 19:52:13,346 - INFO - Calculating metrics for right eye...


2025-04-17 19:52:13,969 - INFO - Calculating metrics for left eye...
2025-04-17 19:52:14,600 - INFO - Finished processing file.
2025-04-17 19:52:14,638 - INFO - Saved calculated data to /home/lrn/Repos/analyze-eye-tracking-data/output/60f931f1-b263-4947-80e6-18869129b673_processed_results.csv
2025-04-17 19:52:14,638 - INFO - Starting animation generation for: 60f931f1-b263-4947-80e6-18869129b673_animation.gif
2025-04-17 19:52:14,646 - INFO - Using 267 frames for animation .
2025-04-17 19:52:14,716 - INFO - Animation.save using <class 'matplotlib.animation.PillowWriter'>
2025-04-17 19:53:21,575 - INFO - Animation saved to /home/lrn/Repos/analyze-eye-tracking-data/output/60f931f1-b263-4947-80e6-18869129b673_animation.gif
2025-04-17 19:53:21,577 - INFO - Finished animation generation for: 60f931f1-b263-4947-80e6-18869129b673_animation.gif
2025-04-17 19:53:21,578 - INFO - --- Finished processing file: 60f931f1-b263-4947-80e6-18869129b673_plr_landmarks.csv ---
2025-04-17 19:53:21,578 - INFO


--- Processing Summary ---
Processed 36 files.
Generated files are located in: /home/lrn/Repos/analyze-eye-tracking-data/output
 - 60f931f1-b263-4947-80e6-18869129b673_plr_landmarks:
   - CSV: 60f931f1-b263-4947-80e6-18869129b673_processed_results.csv
   - GIF: 60f931f1-b263-4947-80e6-18869129b673_animation.gif
 - 52ed40df-14ee-473b-9e71-c539ea24fc44_plr_landmarks:
   - CSV: 52ed40df-14ee-473b-9e71-c539ea24fc44_processed_results.csv
   - GIF: 52ed40df-14ee-473b-9e71-c539ea24fc44_animation.gif
 - 544330a5-9fd0-48b5-9f6d-04a533b33578_plr_landmarks:
   - CSV: 544330a5-9fd0-48b5-9f6d-04a533b33578_processed_results.csv
   - GIF: 544330a5-9fd0-48b5-9f6d-04a533b33578_animation.gif
 - 41fdc7f4-f19a-4058-9241-88c0074e7814_plr_landmarks:
   - CSV: 41fdc7f4-f19a-4058-9241-88c0074e7814_processed_results.csv
   - GIF: 41fdc7f4-f19a-4058-9241-88c0074e7814_animation.gif
 - 55af028a-d7ae-4469-a737-b52fdf75ca24_plr_landmarks:
   - CSV: 55af028a-d7ae-4469-a737-b52fdf75ca24_processed_results.csv
   - GI